<a href="https://colab.research.google.com/github/AlexKressner/WS25_Supply_Chain_Optimierung/blob/main/Fallstudie_Juicy_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG (Network Design)

In [ ]:
! pip install -q pyscipopt

In [ ]:

import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [ ]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

fatal: destination path 'WS25_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [ ]:
! ls

sample_data  WS25_Supply_Chain_Optimierung


In [ ]:
! ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign/

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [ ]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [ ]:
operative_kosten = pd.read_csv(f"{folder}/NetworkDesign_Produktionskosten.csv", sep=";")

In [ ]:
operative_kosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [ ]:
operative_kosten.head()

,Produktionsstandort,Markt,Produktionskosten
0,Charleston,USA,81
1,Charleston,Südamerika,92
2,Charleston,Europa,101
3,Charleston,Asien,130
4,Charleston,Afrika,115


In [ ]:
kapazitaeten = pd.read_csv(f"{folder}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [ ]:
kapazitaeten.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [ ]:
nachfrage = pd.read_csv(f"{folder}/NetworkDesign_Nachfrage.csv", sep=";")

In [ ]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Indexmengen

In [ ]:
I = kapazitaeten.Produktionsstandort.unique()
J = nachfrage.Markt.unique()
A = kapazitaeten.Ausbaustufe.unique()

## Parameter

In [ ]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]

In [ ]:
# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]

In [ ]:
# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
cap

{'klein': 10, 'groß': 20}

In [ ]:
# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
d


{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

## Entscheidungsvariablen

## Zielfunktion

## Nebenbedingungen

## Lösung